# 生成式 AI 入門 (Introduction to Generative AI)

**對應課程**: 李宏毅 2025 Fall GenAI-ML HW1 & HW3

本 notebook 涵蓋生成式 AI 的基礎概念，從傳統機器學習到現代大型語言模型的演進。

## 學習目標
1. 理解生成式 AI 與判別式 AI 的區別
2. 掌握 LLM 的核心架構（GPT vs BERT）
3. 學會 Tokenization 的原理與實作
4. 理解推理參數對生成的影響
5. 實踐基礎 Prompt Engineering

## Part 1: 生成式 AI 概覽

### 1.1 判別式 vs 生成式模型

```
┌─────────────────────────────────────────────────────────────┐
│                     AI 模型分類                              │
├────────────────────────┬────────────────────────────────────┤
│     判別式模型          │          生成式模型                │
│   (Discriminative)     │        (Generative)               │
├────────────────────────┼────────────────────────────────────┤
│ 學習 P(Y|X)            │ 學習 P(X) 或 P(X|Y)               │
│ 輸入 → 類別/數值        │ 條件 → 新資料                     │
├────────────────────────┼────────────────────────────────────┤
│ 範例:                   │ 範例:                             │
│ - 圖像分類              │ - 文本生成 (GPT)                  │
│ - 情感分析              │ - 圖像生成 (Stable Diffusion)     │
│ - 物件偵測              │ - 語音合成 (TTS)                  │
│ - 回歸預測              │ - 程式碼生成 (Codex)              │
└────────────────────────┴────────────────────────────────────┘
```

In [ ]:
# 環境設置
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple, Optional

# 檢查設備
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

### 1.2 生成式 AI 的發展歷程

```
時間軸: 生成式 AI 演進
═══════════════════════════════════════════════════════════════

2014    2017      2018       2020      2022      2023     2024+
  │       │         │          │         │         │        │
  ▼       ▼         ▼          ▼         ▼         ▼        ▼
┌───┐  ┌─────┐  ┌──────┐  ┌──────┐  ┌──────┐  ┌──────┐  ┌─────┐
│GAN│  │Trans│  │ GPT  │  │GPT-3 │  │ChatGP│  │GPT-4 │  │Multi│
│   │  │former│  │ BERT │  │DALL-E│  │  T   │  │Claude│  │modal│
└───┘  └─────┘  └──────┘  └──────┘  └──────┘  └──────┘  └─────┘
  │       │         │          │         │         │        │
  │       │         │          │         │         │        │
圖像    序列到    預訓練    大規模    對話     多模態   AI Agent
生成    序列      語言模型   生成     助手     理解     自主性
```

## Part 2: 語言模型基礎

### 2.1 什麼是語言模型？

語言模型的核心任務：給定前文，預測下一個 token 的機率分布。

$$P(w_t | w_1, w_2, ..., w_{t-1})$$

生成整個序列的機率：
$$P(w_1, w_2, ..., w_T) = \prod_{t=1}^{T} P(w_t | w_1, ..., w_{t-1})$$

In [ ]:
# 簡單的 N-gram 語言模型示範
from collections import defaultdict
import random

class BigramLanguageModel:
    """最簡單的語言模型：Bigram (2-gram)"""
    
    def __init__(self):
        self.bigram_counts = defaultdict(lambda: defaultdict(int))
        self.unigram_counts = defaultdict(int)
    
    def train(self, corpus: List[List[str]]):
        """訓練 bigram 模型"""
        for sentence in corpus:
            # 加入起始和結束標記
            tokens = ['<BOS>'] + sentence + ['<EOS>']
            for i in range(len(tokens) - 1):
                prev_token = tokens[i]
                curr_token = tokens[i + 1]
                self.bigram_counts[prev_token][curr_token] += 1
                self.unigram_counts[prev_token] += 1
    
    def get_probability(self, prev_token: str, curr_token: str) -> float:
        """計算 P(curr_token | prev_token)"""
        if self.unigram_counts[prev_token] == 0:
            return 0.0
        return self.bigram_counts[prev_token][curr_token] / self.unigram_counts[prev_token]
    
    def generate(self, max_length: int = 20) -> List[str]:
        """生成文本"""
        result = []
        current = '<BOS>'
        
        for _ in range(max_length):
            # 取得下一個 token 的機率分布
            next_tokens = self.bigram_counts[current]
            if not next_tokens:
                break
            
            # 依機率抽樣
            tokens = list(next_tokens.keys())
            weights = list(next_tokens.values())
            total = sum(weights)
            probs = [w / total for w in weights]
            
            next_token = random.choices(tokens, probs)[0]
            if next_token == '<EOS>':
                break
            
            result.append(next_token)
            current = next_token
        
        return result

# 訓練範例
corpus = [
    ['I', 'love', 'machine', 'learning'],
    ['I', 'love', 'deep', 'learning'],
    ['machine', 'learning', 'is', 'amazing'],
    ['deep', 'learning', 'is', 'powerful'],
    ['I', 'study', 'AI', 'every', 'day'],
    ['AI', 'is', 'the', 'future'],
]

model = BigramLanguageModel()
model.train(corpus)

# 測試機率計算
print("P('love' | 'I') =", f"{model.get_probability('I', 'love'):.3f}")
print("P('learning' | 'machine') =", f"{model.get_probability('machine', 'learning'):.3f}")

# 生成文本
print("\n生成的句子:")
for i in range(5):
    generated = model.generate()
    print(f"  {i+1}. {' '.join(generated)}")

### 2.2 從 N-gram 到神經網路語言模型

N-gram 的限制：
- 只能捕捉固定長度的上下文
- 詞彙量大時記憶體爆炸
- 無法處理未見過的組合

神經網路語言模型的優勢：
- 學習 word embeddings（詞向量）
- 可變長度上下文（RNN → Transformer）
- 更好的泛化能力

In [ ]:
# 簡單的神經網路語言模型
class SimpleNeuralLM(nn.Module):
    """基於 MLP 的簡單語言模型"""
    
    def __init__(self, vocab_size: int, embed_dim: int = 64, 
                 context_length: int = 3, hidden_dim: int = 128):
        super().__init__()
        self.context_length = context_length
        
        # 詞嵌入層
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # 前饋網路
        self.fc1 = nn.Linear(context_length * embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, x):
        # x: (batch_size, context_length)
        embeds = self.embedding(x)  # (batch_size, context_length, embed_dim)
        embeds = embeds.view(x.size(0), -1)  # 展平
        hidden = F.relu(self.fc1(embeds))
        logits = self.fc2(hidden)  # (batch_size, vocab_size)
        return logits

# 視覺化詞嵌入空間
def visualize_embeddings(model, vocab, title="Word Embeddings"):
    """使用 PCA 視覺化詞嵌入"""
    from sklearn.decomposition import PCA
    
    embeddings = model.embedding.weight.detach().cpu().numpy()
    
    # PCA 降維到 2D
    pca = PCA(n_components=2)
    coords = pca.fit_transform(embeddings)
    
    plt.figure(figsize=(10, 8))
    plt.scatter(coords[:, 0], coords[:, 1], alpha=0.6)
    
    # 標註詞彙
    for i, word in enumerate(vocab):
        plt.annotate(word, (coords[i, 0], coords[i, 1]), fontsize=9)
    
    plt.title(title)
    plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]:.1%})")
    plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]:.1%})")
    plt.grid(True, alpha=0.3)
    plt.show()

# 建立簡單模型
vocab = ['<PAD>', '<BOS>', '<EOS>', 'I', 'love', 'machine', 'deep', 
         'learning', 'is', 'amazing', 'powerful', 'AI', 'study', 'the', 'future', 'every', 'day']
vocab_size = len(vocab)

simple_lm = SimpleNeuralLM(vocab_size=vocab_size, embed_dim=32, context_length=3)
print(f"模型參數量: {sum(p.numel() for p in simple_lm.parameters()):,}")
print(f"\n模型結構:\n{simple_lm}")

## Part 3: GPT vs BERT - 兩種預訓練範式

### 3.1 架構比較

```
┌────────────────────────────────────────────────────────────────┐
│                     GPT vs BERT 比較                           │
├──────────────────────────┬─────────────────────────────────────┤
│          GPT             │               BERT                  │
│   (Decoder-only)         │          (Encoder-only)             │
├──────────────────────────┼─────────────────────────────────────┤
│                          │                                     │
│    ┌───┐                 │         ┌───┐                       │
│    │Out│ ← 只能看到      │         │Out│ ← 可以看到            │
│    └─▲─┘   左側          │         └─▲─┘   雙向                │
│      │                   │           │                         │
│  ┌───┴───┐               │       ┌───┴───┐                     │
│  │Decoder│               │       │Encoder│                     │
│  │ Block │×N             │       │ Block │×N                   │
│  └───┬───┘               │       └───┬───┘                     │
│      │                   │           │                         │
│  [w1][w2][w3]            │       [w1][MASK][w3]                │
│    │   │   │             │         │    │    │                 │
│    ▼   ▼   ▼             │         ▼    ▼    ▼                 │
│   因果注意力             │       雙向注意力                    │
│   (Causal)               │       (Bidirectional)               │
├──────────────────────────┼─────────────────────────────────────┤
│ 預訓練任務:              │ 預訓練任務:                         │
│ Next Token Prediction    │ Masked Language Model (MLM)         │
├──────────────────────────┼─────────────────────────────────────┤
│ 適合任務:                │ 適合任務:                           │
│ - 文本生成               │ - 文本分類                          │
│ - 對話系統               │ - 命名實體識別                      │
│ - 程式碼補全             │ - 問答系統                          │
│ - 翻譯                   │ - 句子相似度                        │
└──────────────────────────┴─────────────────────────────────────┘
```

In [ ]:
# 視覺化因果注意力 vs 雙向注意力
def visualize_attention_masks():
    """視覺化 GPT (causal) vs BERT (bidirectional) 的注意力遮罩"""
    seq_len = 6
    
    # GPT: 因果遮罩 (只能看到自己和左邊)
    causal_mask = torch.tril(torch.ones(seq_len, seq_len))
    
    # BERT: 雙向遮罩 (可以看到所有位置，除了 [MASK])
    bidirectional_mask = torch.ones(seq_len, seq_len)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # GPT 因果注意力
    ax1 = axes[0]
    im1 = ax1.imshow(causal_mask, cmap='Blues', vmin=0, vmax=1)
    ax1.set_title('GPT: Causal Attention Mask\n(只能看到左側)', fontsize=12)
    ax1.set_xlabel('Key Position')
    ax1.set_ylabel('Query Position')
    tokens_gpt = ['I', 'love', 'deep', 'learning', 'and', 'AI']
    ax1.set_xticks(range(seq_len))
    ax1.set_yticks(range(seq_len))
    ax1.set_xticklabels(tokens_gpt, fontsize=9)
    ax1.set_yticklabels(tokens_gpt, fontsize=9)
    
    # 在每個格子標註
    for i in range(seq_len):
        for j in range(seq_len):
            text = '✓' if causal_mask[i, j] == 1 else '✗'
            ax1.text(j, i, text, ha='center', va='center', fontsize=12)
    
    # BERT 雙向注意力
    ax2 = axes[1]
    im2 = ax2.imshow(bidirectional_mask, cmap='Greens', vmin=0, vmax=1)
    ax2.set_title('BERT: Bidirectional Attention Mask\n(可看到全部)', fontsize=12)
    ax2.set_xlabel('Key Position')
    ax2.set_ylabel('Query Position')
    tokens_bert = ['I', 'love', '[MASK]', 'learning', 'and', 'AI']
    ax2.set_xticks(range(seq_len))
    ax2.set_yticks(range(seq_len))
    ax2.set_xticklabels(tokens_bert, fontsize=9)
    ax2.set_yticklabels(tokens_bert, fontsize=9)
    
    for i in range(seq_len):
        for j in range(seq_len):
            ax2.text(j, i, '✓', ha='center', va='center', fontsize=12)
    
    plt.tight_layout()
    plt.show()

visualize_attention_masks()

## Part 4: Tokenization - 文本到數字的橋樑

### 4.1 為什麼需要 Tokenization？

神經網路只能處理數字，因此需要將文本轉換成數字序列。

```
Tokenization 方式比較:
═══════════════════════════════════════════════════════════════

方式          │ 範例: "unhappiness"          │ 優缺點
──────────────┼──────────────────────────────┼───────────────────
字元級        │ [u, n, h, a, p, p, i, n, e,  │ + 詞彙量小
(Character)  │  s, s]                        │ - 序列太長
──────────────┼──────────────────────────────┼───────────────────
詞級         │ [unhappiness] 或 [UNK]       │ + 語義清晰
(Word)       │                               │ - 詞彙量爆炸, OOV
──────────────┼──────────────────────────────┼───────────────────
子詞級       │ [un, happiness] 或           │ + 平衡詞彙量與語義
(Subword)    │ [un, happ, iness]            │ + 處理未見詞
             │                               │ = BPE, WordPiece
```

In [ ]:
# BPE (Byte Pair Encoding) 演算法簡化實作
class SimpleBPE:
    """簡化版 BPE tokenizer"""
    
    def __init__(self, vocab_size: int = 100):
        self.vocab_size = vocab_size
        self.merges = {}  # 合併規則
        self.vocab = {}   # 詞彙表
    
    def _get_stats(self, corpus: List[List[str]]) -> Dict[Tuple[str, str], int]:
        """統計相鄰 token pair 頻率"""
        pairs = defaultdict(int)
        for word in corpus:
            for i in range(len(word) - 1):
                pairs[(word[i], word[i+1])] += 1
        return pairs
    
    def _merge(self, corpus: List[List[str]], pair: Tuple[str, str]) -> List[List[str]]:
        """合併指定的 pair"""
        new_corpus = []
        bigram = ' '.join(pair)
        replacement = ''.join(pair)
        
        for word in corpus:
            new_word = []
            i = 0
            while i < len(word):
                if i < len(word) - 1 and word[i] == pair[0] and word[i+1] == pair[1]:
                    new_word.append(replacement)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_corpus.append(new_word)
        return new_corpus
    
    def train(self, texts: List[str], num_merges: int = 10):
        """訓練 BPE"""
        # 初始化：將每個字元分開，並加上詞尾標記
        corpus = [list(word) + ['</w>'] for text in texts for word in text.split()]
        
        # 建立初始詞彙表
        self.vocab = set()
        for word in corpus:
            self.vocab.update(word)
        
        print(f"初始詞彙量: {len(self.vocab)}")
        print(f"初始詞彙: {sorted(self.vocab)}")
        print("\n開始 BPE 合併:")
        
        # 執行合併
        for i in range(num_merges):
            pairs = self._get_stats(corpus)
            if not pairs:
                break
            
            # 找出最頻繁的 pair
            best_pair = max(pairs, key=pairs.get)
            merged = ''.join(best_pair)
            
            print(f"  {i+1}. 合併 {best_pair} → '{merged}' (頻率: {pairs[best_pair]})")
            
            # 執行合併
            corpus = self._merge(corpus, best_pair)
            self.merges[best_pair] = merged
            self.vocab.add(merged)
        
        print(f"\n最終詞彙量: {len(self.vocab)}")
        return corpus

# BPE 訓練示範
texts = [
    "low lower lowest",
    "new newer newest",
    "learning learned learner"
]

bpe = SimpleBPE()
result = bpe.train(texts, num_merges=15)

print("\n分詞結果:")
for i, word in enumerate(result[:6]):
    print(f"  {word}")

In [ ]:
# 使用 Hugging Face transformers 的 Tokenizer
try:
    from transformers import AutoTokenizer
    
    # 載入 GPT-2 tokenizer
    tokenizer = AutoTokenizer.from_pretrained('gpt2')
    
    # 測試不同文本的 tokenization
    test_texts = [
        "Hello, how are you?",
        "Machine learning is fascinating!",
        "深度學習很有趣",  # 中文
        "unhappiness is temporary",
        "The quick brown fox jumps over the lazy dog."
    ]
    
    print("GPT-2 Tokenization 結果:")
    print("=" * 60)
    for text in test_texts:
        tokens = tokenizer.tokenize(text)
        token_ids = tokenizer.encode(text)
        print(f"\n原文: {text}")
        print(f"Tokens ({len(tokens)}): {tokens}")
        print(f"Token IDs: {token_ids}")
    
    # 詞彙表大小
    print(f"\nGPT-2 詞彙表大小: {tokenizer.vocab_size:,}")
    
except ImportError:
    print("請安裝 transformers: pip install transformers")

## Part 5: 推理參數 - 控制生成行為

### 5.1 Temperature（溫度）

Temperature 控制機率分布的「銳利度」：

$$P_i = \frac{\exp(z_i / T)}{\sum_j \exp(z_j / T)}$$

- **T = 0**: 最確定性（greedy decoding）
- **T = 1**: 原始機率分布
- **T > 1**: 更均勻、更隨機
- **T < 1**: 更集中、更確定

In [ ]:
# Temperature 效果視覺化
def visualize_temperature_effect():
    """展示不同 temperature 對機率分布的影響"""
    # 原始 logits (假設有 5 個候選詞)
    logits = torch.tensor([2.0, 1.5, 0.5, 0.2, -0.5])
    vocab = ['the', 'a', 'an', 'this', 'that']
    
    temperatures = [0.1, 0.5, 1.0, 2.0, 5.0]
    
    fig, axes = plt.subplots(1, len(temperatures), figsize=(15, 4))
    
    for ax, T in zip(axes, temperatures):
        if T == 0.1:  # 近似 greedy
            probs = F.softmax(logits / T, dim=0).numpy()
        else:
            probs = F.softmax(logits / T, dim=0).numpy()
        
        colors = plt.cm.Blues(probs / probs.max())
        bars = ax.bar(vocab, probs, color=colors, edgecolor='navy')
        ax.set_title(f'T = {T}', fontsize=12)
        ax.set_ylim(0, 1)
        ax.set_ylabel('Probability' if T == 0.1 else '')
        
        # 標註機率值
        for bar, p in zip(bars, probs):
            if p > 0.01:
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                       f'{p:.2f}', ha='center', fontsize=9)
    
    plt.suptitle('Temperature 對機率分布的影響', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

visualize_temperature_effect()

In [ ]:
# Top-K 和 Top-P (Nucleus) Sampling 比較
def demonstrate_sampling_strategies():
    """展示不同採樣策略"""
    # 模擬的機率分布（長尾分布）
    vocab_size = 20
    probs = F.softmax(torch.randn(vocab_size) * 2, dim=0).sort(descending=True)[0]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 1. Top-K Sampling (K=5)
    ax1 = axes[0]
    k = 5
    colors1 = ['steelblue' if i < k else 'lightgray' for i in range(vocab_size)]
    ax1.bar(range(vocab_size), probs.numpy(), color=colors1, edgecolor='black', linewidth=0.5)
    ax1.axvline(x=k-0.5, color='red', linestyle='--', label=f'Top-{k} cutoff')
    ax1.set_title(f'Top-K Sampling (K={k})', fontsize=12)
    ax1.set_xlabel('Token (sorted by prob)')
    ax1.set_ylabel('Probability')
    ax1.legend()
    
    # 2. Top-P (Nucleus) Sampling (P=0.9)
    ax2 = axes[1]
    p_threshold = 0.9
    cumsum = probs.cumsum(0).numpy()
    cutoff_idx = (cumsum < p_threshold).sum() + 1
    colors2 = ['forestgreen' if i < cutoff_idx else 'lightgray' for i in range(vocab_size)]
    ax2.bar(range(vocab_size), probs.numpy(), color=colors2, edgecolor='black', linewidth=0.5)
    ax2.axvline(x=cutoff_idx-0.5, color='red', linestyle='--', label=f'Top-P={p_threshold} cutoff')
    ax2.set_title(f'Top-P (Nucleus) Sampling (P={p_threshold})', fontsize=12)
    ax2.set_xlabel('Token (sorted by prob)')
    ax2.legend()
    
    # 3. 累積機率曲線
    ax3 = axes[2]
    ax3.plot(range(vocab_size), cumsum, 'o-', color='purple', markersize=4)
    ax3.axhline(y=p_threshold, color='red', linestyle='--', label=f'P={p_threshold}')
    ax3.fill_between(range(cutoff_idx), cumsum[:cutoff_idx], alpha=0.3, color='green')
    ax3.set_title('Cumulative Probability', fontsize=12)
    ax3.set_xlabel('Token (sorted by prob)')
    ax3.set_ylabel('Cumulative Prob')
    ax3.legend()
    ax3.set_ylim(0, 1.05)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Top-K (K=5): 保留前 5 個 token")
    print(f"Top-P (P=0.9): 保留累積機率達到 90% 的 token (本例 {cutoff_idx} 個)")

demonstrate_sampling_strategies()

### 5.2 生成參數總結

```
┌────────────────────────────────────────────────────────────────┐
│                    生成參數速查表                               │
├───────────────┬────────────────────────────────────────────────┤
│ 參數           │ 說明                                           │
├───────────────┼────────────────────────────────────────────────┤
│ temperature   │ 控制隨機性，低=確定，高=隨機                    │
│ top_k         │ 只從機率最高的 K 個 token 中採樣                │
│ top_p         │ 只從累積機率達到 P 的 token 中採樣              │
│ max_tokens    │ 生成的最大 token 數量                           │
│ stop          │ 停止生成的特定序列                              │
│ frequency_penalty │ 降低重複 token 的機率                       │
│ presence_penalty  │ 鼓勵生成新的 token                          │
├───────────────┼────────────────────────────────────────────────┤
│               │ 建議配置                                        │
├───────────────┼────────────────────────────────────────────────┤
│ 創意寫作       │ T=0.8-1.0, top_p=0.9                           │
│ 程式碼生成     │ T=0.2-0.4, top_p=0.95                          │
│ 事實問答       │ T=0.0-0.2 (greedy)                             │
│ 翻譯           │ T=0.3-0.5                                      │
└───────────────┴────────────────────────────────────────────────┘
```

## Part 6: Prompt Engineering 基礎

### 6.1 Prompt 設計原則

```
┌─────────────────────────────────────────────────────────────┐
│                 Prompt Engineering 最佳實踐                   │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. 明確指令 (Be Specific)                                  │
│     ❌ "寫一篇文章"                                          │
│     ✅ "寫一篇 300 字的科普文章，主題是量子計算，             │
│         目標讀者是高中生"                                    │
│                                                             │
│  2. 提供範例 (Few-shot)                                     │
│     給出輸入-輸出的範例，讓模型學習格式                       │
│                                                             │
│  3. 角色設定 (System Prompt)                                │
│     "你是一位專業的資料科學家..."                            │
│                                                             │
│  4. 結構化輸出 (Output Format)                              │
│     要求 JSON、Markdown、或特定格式                          │
│                                                             │
│  5. 思維鏈 (Chain-of-Thought)                               │
│     "請一步一步思考..."、"讓我們分析一下..."                  │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Prompt 模板範例
prompt_templates = {
    "zero_shot": """請將以下英文翻譯成中文：
{text}

中文翻譯：""",
    
    "few_shot": """請將英文翻譯成中文。

範例 1:
英文: Hello, how are you?
中文: 你好，你好嗎？

範例 2:
英文: Machine learning is amazing.
中文: 機器學習太神奇了。

現在請翻譯:
英文: {text}
中文:""",
    
    "chain_of_thought": """問題：{question}

請一步一步思考這個問題：

步驟 1: 理解問題
步驟 2: 分析關鍵資訊
步驟 3: 推理過程
步驟 4: 得出結論

讓我們開始：""",
    
    "role_play": """你是一位經驗豐富的 Python 程式設計師，專精於資料科學和機器學習。
你的回答應該：
1. 提供可執行的程式碼
2. 包含詳細的註解
3. 解釋程式碼的工作原理

使用者問題：{question}

你的回答："""
}

# 展示模板
for name, template in prompt_templates.items():
    print(f"\n{'='*60}")
    print(f"模板類型: {name}")
    print('='*60)
    print(template.format(text="Deep learning is powerful.", 
                         question="如何用 Python 實作快速排序？")[:300] + "...")

## Part 7: 實作練習 - 使用 transformers 進行文本生成

In [ ]:
# 使用 GPT-2 進行文本生成
try:
    from transformers import GPT2LMHeadModel, GPT2Tokenizer
    import torch
    
    # 載入模型和 tokenizer
    print("載入 GPT-2 模型...")
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.eval()
    
    # 移到 GPU（如果可用）
    model = model.to(device)
    print(f"模型載入完成，使用 {device}")
    print(f"模型參數量: {sum(p.numel() for p in model.parameters()):,}")
    
    def generate_text(prompt: str, max_length: int = 100, 
                     temperature: float = 1.0, top_k: int = 50, 
                     top_p: float = 0.95, num_return_sequences: int = 1):
        """使用 GPT-2 生成文本"""
        # Tokenize 輸入
        input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
        
        # 設定 attention mask
        attention_mask = torch.ones_like(input_ids)
        
        # 生成
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                num_return_sequences=num_return_sequences,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # 解碼
        generated_texts = []
        for output in outputs:
            text = tokenizer.decode(output, skip_special_tokens=True)
            generated_texts.append(text)
        
        return generated_texts
    
    # 測試生成
    prompt = "Artificial intelligence will"
    print(f"\n輸入提示: '{prompt}'")
    print("\n生成結果 (temperature=0.7):")
    print("-" * 50)
    
    results = generate_text(prompt, max_length=80, temperature=0.7, num_return_sequences=3)
    for i, text in enumerate(results, 1):
        print(f"\n[{i}] {text}")
    
except ImportError:
    print("請安裝 transformers: pip install transformers")

In [ ]:
# 比較不同 temperature 的生成效果
try:
    prompt = "The future of technology is"
    temperatures = [0.2, 0.7, 1.0, 1.5]
    
    print(f"輸入提示: '{prompt}'")
    print("\n比較不同 Temperature 的生成效果:")
    print("=" * 70)
    
    for temp in temperatures:
        print(f"\n[Temperature = {temp}]")
        result = generate_text(prompt, max_length=60, temperature=temp, num_return_sequences=1)
        print(f"  {result[0]}")
        
except NameError:
    print("請先執行上方的模型載入程式碼")

## Part 8: 練習題

### Exercise 1: 實作 Greedy Decoding

實作一個簡單的 greedy decoding 函數，每次選擇機率最高的 token。

In [ ]:
def greedy_decode(model, tokenizer, prompt: str, max_new_tokens: int = 50):
    """
    實作 Greedy Decoding
    
    Args:
        model: GPT-2 模型
        tokenizer: tokenizer
        prompt: 輸入提示
        max_new_tokens: 最大生成 token 數
    
    Returns:
        生成的文本
    """
    # TODO: 實作 greedy decoding
    # 提示：
    # 1. 將 prompt 轉換成 token ids
    # 2. 迴圈生成新 token
    # 3. 每次取機率最高的 token (argmax)
    # 4. 如果生成 EOS token 則停止
    
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    generated = input_ids.clone()
    
    model.eval()
    with torch.no_grad():
        for _ in range(max_new_tokens):
            # 取得模型輸出
            outputs = model(generated)
            logits = outputs.logits  # (batch, seq_len, vocab_size)
            
            # 取最後一個位置的 logits
            next_token_logits = logits[:, -1, :]  # (batch, vocab_size)
            
            # Greedy: 選擇機率最高的 token
            next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
            
            # 加入到生成序列
            generated = torch.cat([generated, next_token], dim=-1)
            
            # 檢查是否生成 EOS
            if next_token.item() == tokenizer.eos_token_id:
                break
    
    return tokenizer.decode(generated[0], skip_special_tokens=True)

# 測試
try:
    prompt = "Machine learning is"
    result = greedy_decode(model, tokenizer, prompt, max_new_tokens=30)
    print(f"Prompt: {prompt}")
    print(f"Greedy output: {result}")
except NameError:
    print("請先載入 GPT-2 模型")

### Exercise 2: Temperature Sampling 實作

修改上面的 greedy decoding，加入 temperature 參數進行採樣。

In [ ]:
def temperature_sampling(model, tokenizer, prompt: str, 
                         max_new_tokens: int = 50, temperature: float = 1.0):
    """
    實作 Temperature Sampling
    
    Args:
        model: GPT-2 模型
        tokenizer: tokenizer
        prompt: 輸入提示
        max_new_tokens: 最大生成 token 數
        temperature: 溫度參數 (0 < T <= inf)
    
    Returns:
        生成的文本
    """
    # TODO: 實作 temperature sampling
    # 提示：
    # 1. 對 logits 除以 temperature
    # 2. 使用 softmax 得到機率分布
    # 3. 使用 torch.multinomial 進行採樣
    
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    generated = input_ids.clone()
    
    model.eval()
    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(generated)
            logits = outputs.logits[:, -1, :]  # (batch, vocab_size)
            
            # 應用 temperature
            scaled_logits = logits / temperature
            
            # 轉換成機率
            probs = F.softmax(scaled_logits, dim=-1)
            
            # 採樣
            next_token = torch.multinomial(probs, num_samples=1)
            
            generated = torch.cat([generated, next_token], dim=-1)
            
            if next_token.item() == tokenizer.eos_token_id:
                break
    
    return tokenizer.decode(generated[0], skip_special_tokens=True)

# 測試不同 temperature
try:
    prompt = "The secret to happiness is"
    print(f"Prompt: {prompt}\n")
    
    for temp in [0.3, 0.7, 1.0, 1.5]:
        result = temperature_sampling(model, tokenizer, prompt, 
                                      max_new_tokens=25, temperature=temp)
        print(f"T={temp}: {result}\n")
except NameError:
    print("請先載入 GPT-2 模型")

### Exercise 3: 實作 Top-K 和 Top-P Sampling

結合 Top-K 和 Top-P 進行更精細的生成控制。

In [ ]:
def top_k_top_p_sampling(model, tokenizer, prompt: str,
                         max_new_tokens: int = 50,
                         temperature: float = 1.0,
                         top_k: int = 50,
                         top_p: float = 0.95):
    """
    實作 Top-K + Top-P (Nucleus) Sampling
    
    Args:
        model: GPT-2 模型
        tokenizer: tokenizer
        prompt: 輸入提示
        max_new_tokens: 最大生成 token 數
        temperature: 溫度參數
        top_k: 只保留前 K 個機率最高的 token
        top_p: 只保留累積機率達到 P 的 token
    
    Returns:
        生成的文本
    """
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    generated = input_ids.clone()
    
    model.eval()
    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(generated)
            logits = outputs.logits[:, -1, :] / temperature
            
            # Top-K filtering
            if top_k > 0:
                # 保留前 K 個，其他設為 -inf
                top_k_logits, top_k_indices = torch.topk(logits, top_k)
                logits_filtered = torch.full_like(logits, float('-inf'))
                logits_filtered.scatter_(1, top_k_indices, top_k_logits)
                logits = logits_filtered
            
            # Top-P (Nucleus) filtering
            if top_p < 1.0:
                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
                
                # 移除累積機率超過 top_p 的 token
                sorted_indices_to_remove = cumulative_probs > top_p
                # 保留第一個超過的（確保至少有一個 token）
                sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
                sorted_indices_to_remove[:, 0] = False
                
                indices_to_remove = sorted_indices_to_remove.scatter(
                    1, sorted_indices, sorted_indices_to_remove
                )
                logits[indices_to_remove] = float('-inf')
            
            # 採樣
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            
            generated = torch.cat([generated, next_token], dim=-1)
            
            if next_token.item() == tokenizer.eos_token_id:
                break
    
    return tokenizer.decode(generated[0], skip_special_tokens=True)

# 測試
try:
    prompt = "In the year 2050, humans will"
    print(f"Prompt: {prompt}\n")
    
    # 不同配置比較
    configs = [
        {"temperature": 0.7, "top_k": 0, "top_p": 1.0, "name": "Only Temperature"},
        {"temperature": 0.7, "top_k": 50, "top_p": 1.0, "name": "Top-K=50"},
        {"temperature": 0.7, "top_k": 0, "top_p": 0.9, "name": "Top-P=0.9"},
        {"temperature": 0.7, "top_k": 50, "top_p": 0.9, "name": "Top-K=50 + Top-P=0.9"},
    ]
    
    for cfg in configs:
        result = top_k_top_p_sampling(
            model, tokenizer, prompt,
            max_new_tokens=30,
            temperature=cfg["temperature"],
            top_k=cfg["top_k"],
            top_p=cfg["top_p"]
        )
        print(f"[{cfg['name']}]")
        print(f"  {result}\n")
        
except NameError:
    print("請先載入 GPT-2 模型")

## 總結

### 本 Notebook 涵蓋的重點

```
┌─────────────────────────────────────────────────────────────┐
│                    生成式 AI 入門總結                        │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  1. 基礎概念                                                │
│     • 判別式 vs 生成式模型                                   │
│     • 語言模型的數學定義                                     │
│     • N-gram → 神經網路語言模型演進                          │
│                                                             │
│  2. 架構理解                                                │
│     • GPT (Decoder-only, Causal)                           │
│     • BERT (Encoder-only, Bidirectional)                   │
│     • 預訓練任務差異                                        │
│                                                             │
│  3. Tokenization                                           │
│     • 字元級 vs 詞級 vs 子詞級                               │
│     • BPE 演算法                                            │
│     • 實際 tokenizer 使用                                   │
│                                                             │
│  4. 生成控制                                                │
│     • Temperature                                          │
│     • Top-K Sampling                                       │
│     • Top-P (Nucleus) Sampling                             │
│                                                             │
│  5. Prompt Engineering                                     │
│     • Zero-shot / Few-shot                                 │
│     • Chain-of-Thought                                     │
│     • 角色設定                                              │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### 下一步學習

- **RAG 系統**: `ai_agents/rag_basic.ipynb`
- **LLM 微調**: `language_models/llm_finetuning.ipynb`
- **RLHF**: `reinforcement_learning/rlhf_alignment.ipynb`

## 參考資源

### 課程
- [李宏毅 2025 Fall GenAI-ML](https://speech.ee.ntu.edu.tw/~hylee/GenAI-ML/2025-fall.php)

### 論文
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762) - Transformer
- [Language Models are Unsupervised Multitask Learners](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) - GPT-2
- [BERT: Pre-training of Deep Bidirectional Transformers](https://arxiv.org/abs/1810.04805)

### 工具
- [Hugging Face Transformers](https://huggingface.co/docs/transformers/)
- [OpenAI Tokenizer](https://platform.openai.com/tokenizer)